In [29]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

dist = pd.read_csv('../data/times v4.csv')
le = sklearn.preprocessing.LabelEncoder()
le.fit(dist['Origin_tid'])
dist['from_int'] = le.transform(dist['Origin_tid'])
dist['to_int'] = le.transform(dist['Destination_tid'])

config = {'cnt_terminals': dist['from_int'].max() + 1,
          'persent_day_income': 0.02 / 365,
          'terminal_service_cost': {'min': 100, 'persent': 0.01},
          'max_terminal_money': 1000000,
          'max_not_service_days': 14,
          'armored_car_day_cost': 20000,
          'work_time': 10 * 60,
          'service_time': 10}

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    total_count = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_count = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            route_count += 1
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        route_count -= 1
        total_count += route_count
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        print(f'GOES THROUGH {route_count}')
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))
    print(f'GO THROUGH {total_count}')

inf = int(1e4)
cnt_terminals = config['cnt_terminals']
distance_matrix = np.ones((cnt_terminals + 2, cnt_terminals + 2)) * inf
for i, j, w in zip(dist['from_int'], dist['to_int'], dist['Total_Time']):
    distance_matrix[i + 1, j + 1] = w + config['service_time']
    
for i in range(1, cnt_terminals + 1):
    distance_matrix[i, 0] = inf
    distance_matrix[0, i] = config['service_time']
    distance_matrix[i, i] = 0
    distance_matrix[i, cnt_terminals + 1] = 0
    distance_matrix[cnt_terminals + 1, i] = inf
    
distance_matrix[0, cnt_terminals + 1] = 0
distance_matrix[cnt_terminals + 1, 0] = inf
distance_matrix = distance_matrix.astype(int)

penalties = [1 for i in range(cnt_terminals + 2)]
penalties[0] = 0
penalties[-1] = 0

distance_matrix = distance_matrix.astype(int)

In [30]:
vrp_data = {'distance_matrix': distance_matrix,
            'num_vehicles': 5,
            'num_locations': cnt_terminals + 2,
            'depot': 0}

vrp_data['starts'] = [0] * vrp_data['num_vehicles']
vrp_data['ends'] = [int(cnt_terminals + 1)] * vrp_data['num_vehicles']

In [32]:
manager = pywrapcp.RoutingIndexManager(len(vrp_data['distance_matrix']),
                                   vrp_data['num_vehicles'], vrp_data['starts'], vrp_data['ends'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

# Define cost of each arc.

def distance_callback(from_index, to_index):
    """Returns the manhattan distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return vrp_data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    config['work_time'],  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Allow to drop nodes.
penalty = inf
for node in range(1, len(vrp_data['distance_matrix']) - 1):
    routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
search_parameters.time_limit.seconds = 5

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(vrp_data, manager, routing, solution)
else:
    print('bad')

Objective: 13920062853
Route for vehicle 0:
 0 ->  1623 ->  1573 ->  1102 ->  205 ->  917 ->  399 ->  950 ->  1303 ->  958 ->  640 ->  1318 ->  1153 ->  599 ->  915 ->  1076 ->  1452 ->  957 ->  1234 ->  81 ->  1572 ->  1235 ->  836 ->  484 ->  1320 ->  58 ->  607 ->  839 ->  79 ->  841 ->  45 ->  1570 ->  43 ->  837 ->  1135 ->  73 ->  865 ->  858 ->  1228 ->  855 ->  718 ->  787 ->  217 ->  520 ->  1330 ->  1418 ->  216 ->  1510 ->  218 ->  740 ->  1385 -> 1631
Distance of the route: 599m

GOES THROUGH 50
Route for vehicle 1:
 0 ->  1625 ->  532 ->  876 ->  890 ->  1515 ->  1547 ->  912 ->  1265 ->  1138 ->  595 ->  1403 ->  1437 ->  1554 ->  1461 ->  515 ->  883 ->  261 ->  1431 ->  897 ->  1095 ->  1254 ->  1390 ->  1618 ->  617 ->  1387 ->  1562 ->  1134 ->  1493 ->  213 ->  1306 ->  871 ->  609 ->  1082 ->  1617 ->  1559 ->  1458 ->  1479 ->  885 ->  57 ->  537 ->  1289 ->  206 ->  51 ->  1474 ->  582 ->  579 ->  618 ->  893 -> 1631
Distance of the route: 585m

GOES THROUGH 48
Ro